In [ ]:
import json
import csv
import math
import os
import base64
import imageio
import cProfile
import numpy as np
import pandas as pd
import plotly.io as pio
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm
from IPython import display
from joblib import Parallel, delayed
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from matplotlib.patches import Wedge

# Load Parameters & Data

In [ ]:
# path = "Runs/"
# folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
# latest_folder = max(folders, key=lambda f: os.path.getctime(os.path.join(path, f)))

# path = os.path.join(path, latest_folder+"/")
# print(path)
path=r"Runs\High Density\CRun\Thu_Jan_18_23_47_35_2024"

In [ ]:
def save_param(path):
    csv_file_path=path+"/params.csv"
    # Open the CSV file
    with open(csv_file_path, mode='r') as infile:
        reader = csv.reader(infile)
        mydict = {rows[0]:rows[1] for rows in reader}
    return mydict

def load_data(path):
    with open(path+'/particle_positions.json', 'r') as file:
        data = json.load(file)
    return data

variable_dict=save_param(path)

r=float(variable_dict["sigma"])/2.0
numParticles=int(variable_dict["numParticles"])
boxSize=float(variable_dict["boxSize"])
timestep=float(variable_dict["timestep"])
dataCompression=int(variable_dict["dataCompression"])
theta=(float(variable_dict["theta"])) 

data = load_data(path)
particleData = np.array(data)

print(variable_dict)

# Minimum Image Convention Negator

In [ ]:
def min_image_convention(dx, box_size):
    # Apply minimum image convention to get the shortest distance
    dx -= np.round(dx / box_size) * box_size
    return dx

In [ ]:
marker_shapes = {
    15: 'o',   # Circle
    20: '^',   # Triangle
    25: 's',   # Square
    30: 'D',   # Diamond
    35: 'p',   # Pentagon
    40: '*',   # Star
    45: 'v',   # Inverted triangle
    60: '<',   # Left-pointing triangle
    90: '>',   # Right-pointing triangle
    }
cmap = plt.get_cmap('tab20')  # You can choose a different colormap


# Plot Energy

In [ ]:
def energy_plotly_plotter(xlabel, ylabel):
    size=particleData.shape[0]
    legend_names = ["Kinetic Energy", "Potential Energy", "Total Energy"]
    colors = ['blue', 'green', 'red']
    energies=[0,0,0]

    # Calculate  kinetic, potential and total energy per particle for each time step
    energies[0] = np.average(0.5 * np.sum( particleData[:, :, 2:4] ** 2, axis=2), axis=1)
    energies[1] = particleData[:,-1, -1]  #last particle last column contains the avergae PE per particle for that timestep
    energies[2] = energies[0] + energies[1]

    # Create Plotly figure
    fig = go.Figure()
    time = [i*dataCompression*timestep for i in range(size-1)]
    for i in range(3): fig.add_trace(go.Scatter(x=time, y=energies[i], mode='lines', name=legend_names[i], line=dict(color=colors[i])))

    fig.update_layout(title="Energy Plot", xaxis_title=xlabel, yaxis_title=ylabel, width=1200, height=600)
    pio.write_html(fig, path+'/tempi.html')
    fig.show()

energy_plotly_plotter("Time", "Energy per Particle")

# Final Frame Plotter

In [ ]:
def FinalSysImagePlotter(particle_data, box_size):
    # Create a figure and axis for the Plot
    fig, ax = plt.subplots()
    ax.set_xlim(0, box_size)
    ax.set_ylim(0, box_size)
    
    for x, y, vx, vy, phi, _ in particle_data[-1]:
        circle = plt.Circle((x, y), radius=r, linewidth=0)
        ax.add_patch(circle)

    # Set axis labels and title
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.title('Final Image of System')
    
    # Save and show the snapshot
    plt.savefig(path + "/FinalFrame.png") 
    plt.show()

# Plots the final system image
FinalSysImagePlotter(particleData, boxSize)


# Animation

In [ ]:
def generate_frames_parallel(skipped_particle_data, frame, box_size, path):
    plt.figure()
    plt.xlim(0, box_size)
    plt.ylim(0, box_size)
    frame_path = os.path.join(path, f'/frame_{frame}.png')

    # Plot the particles at the given frame as circles
    for x, y, _, _, _, _ in skipped_particle_data[frame]:
        circle = plt.Circle((x, y), radius=r, linewidth=0)
        # add an if statement to change the color for the last particle in the list
        plt.gca().add_patch(circle)
    frame_plt = plt
    frame_plt.savefig(frame_path)
    plt.close()

def show_gif(fname):
    with open(fname, 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')

    gif_html = f'<img src="data:image/gif;base64,{b64}" />'
    link_html = f'<a href="{fname}" target="_blank">Click here for the GIF</a>'

    return display.HTML(f'{gif_html}<br>{link_html}')

def animator(particle_data, box_size, path, skip):
    skipped_particle_data = particle_data[::skip]
    num_frames = skipped_particle_data.shape[0]
    # Use joblib for parallel execution
    Parallel(n_jobs=-1)(delayed(generate_frames_parallel)(skipped_particle_data, frame, box_size, path) for frame in tqdm(range(num_frames)))

    # Combine frames into a GIF using imageio
    with imageio.get_writer(path+'/animation.gif', duration=0.1) as writer:
        for frame in range(num_frames):
            frame_path = os.path.join(path, f'/frame_{frame}.png')
            image = imageio.imread(frame_path)
            writer.append_data(image)
            
    # Delete individual PNG files
    for frame in range(num_frames):
        frame_path = os.path.join(path, f'/frame_{frame}.png')
        os.remove(frame_path)
    show_gif(path+'/animation.gif')
animator(particleData, boxSize, path, 5)

In [ ]:
num_frames = len(particle_data)
def update(frame, box_size):
    # ?Function to update each frame
    plt.figure()
    plt.xlim(0, box_size)
    plt.ylim(0, box_size)

    positions = particle_data[frame, :, :]
    corner_positions=positions[[0,-1, 9, -10, 73],:]
    # Plot the particles at the given frame as circles
    try:
        for x, y, _, _, phi, _ in particle_data[frame]:
            circle = plt.Circle((x, y), radius=r, linewidth=0)
            # add an if statement to change the color for the last particle in the list
            sector = Wedge((x, y), 6, np.degrees(phi) - theta, np.degrees(phi) + theta, ec='black')
            plt.gca().add_patch(sector)
            plt.gca().add_patch(circle)
        for x, y, vx, vy, phi, _ in corner_positions:
            sector = Wedge((x, y), 6, np.degrees(phi) - theta, np.degrees(phi) + theta, ec='black', fc='red')
            plt.gca().add_patch(sector)
    except:
        plt.gca().set_facecolor('black')  # Add this line to make the background black
    return plt

def generate_frames_parallel(frame, box_size):
    frame_plt = update(frame, box_size)
    frame_path = os.path.join(path, f'frame_{frame}.png')
    frame_plt.savefig(frame_path)
    plt.close()

# Use joblib for parallel execution
Parallel(n_jobs=-1)(delayed(generate_frames_parallel)(frame, boxSize) for frame in tqdm(range(num_frames)))

# ?Combine frames into a GIF using imageio
with imageio.get_writer(path+'animation.gif', duration=0.1) as writer:
    for frame in range(num_frames):
        frame_path = os.path.join(path, f'frame_{frame}.png')
        image = imageio.imread(frame_path)
        writer.append_data(image)
    

# ?Delete individual PNG files
for frame in range(num_frames):
    frame_path = os.path.join(path, f'frame_{frame}.png')
    os.remove(frame_path)

In [ ]:
phi = particleData[:1000, :, -2]
dphi = np.diff(phi, axis=0)
print (dphi.shape)
average_phi = np.average(np.abs(dphi), axis=0)
# print the position of the minimum value of average phi
print(np.argmin(average_phi))
# plot the average phi
plt.plot(average_phi)
plt.xlabel('Particles')
plt.ylabel('Average Orientation deviation')
plt.title('Average Orientation deviation vs particles')
plt.savefig(path + "/AveragePhi.png")
plt.show()

# Mean Squared Displacement

In [ ]:
def MSD_particle(i, positions, box_size, size):
    n = np.zeros((size - 1, 2))
    result = []
    for j in range(math.floor(size - 1.0)):
        d = positions[j + 1:, i] - positions[:-j - 1, i]
        d -= np.round((d - n) / (box_size)) * box_size
        n = d[:-1]
        result.append(np.mean((np.linalg.norm(d, axis=1))**2))
    return result



def MSD(particle_data, box_size, time_step, data_compression, skip):
    positions = particle_data[::skip, :, :2]
    size=positions.shape[0]
    num_particles=positions.shape[1]
    MSDPerParticle = np.zeros((num_particles, size - 1))

    # Create a nested for-loop running through each particle and all possible gap values to create the MSD Matrix
    MSDPerParticle=Parallel(n_jobs=-1)(delayed(MSD_particle)(i, positions, box_size, size) for i in tqdm(range(num_particles)))
    MSDPerParticle = np.array(MSDPerParticle)
    
    # Calculate the mean and standard deviation of the MSD values
    averageMSDPerGap=[np.mean(MSDPerParticle[:, j]) for j in range(size-1)]
    stdMSDPerGap=[np.std(MSDPerParticle[:, j]) for j in range(size-1)]

    x=[i*data_compression*time_step*skip for i in range(math.floor(size-1.0))]
    y=averageMSDPerGap[:len(x)]
    dy=stdMSDPerGap[:len(x)]
    return x, y, dy
x,y,dy=MSD(particleData, boxSize, timestep, dataCompression, 5)


In [ ]:
def plot_MSD(x,y, theta, path):
    color = cmap(theta/90)  # Use theta to get a unique color from the colormap
    theta_marker = marker_shapes.get(round(theta), 'o')  # Default to circle if theta not in the dictionary
    #fit and plot the data with an exponential function and print the fit values
    def func(v, a, b, c):
        # return a * np.exp(np.array(v)/b) + c
        return b * np.array(v) + c
    popt, pcov = curve_fit(func, x, y, p0=[1, 100, 0])
    # plt.plot(x, func(x, *popt), label=f'Fit:'" a="+str(round(popt[0], 2))+" b="+str(round(popt[1], 2))+" c="+str(round(popt[2], 2)), color=color)
    print (popt)
    
    # plt.plot(x, y, label=f'Vission Angle \u03B8={round(theta)}\u00B0', marker='o', color=color, markersize=1)
    plt.scatter(x, y, label=f'Vission Angle \u03B8={round(theta)}\u00B0', marker=theta_marker, color=color, s=2)
    plt.yscale('log')
    plt.xscale('log')
    plt.xticks(fontsize=8)
    plt.xticks(fontsize=8)
    plt.grid(True)
    plt.xlabel("Time Gap")
    plt.ylabel("MSD")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=7)
    plt.title("Mean Squared Displacement")
    plt.tight_layout()
    # Set the figure size (adjust these values as needed)
    fig = plt.gcf()
    fig.set_size_inches(10, 6)
    plt.savefig(path+"/MSD.png", bbox_inches = 'tight')
    # return popt and theta as a dictionary for simm plot
    return popt

plot_MSD(x,y, theta, path) 

# Auto-Correlation Function

In [ ]:
def autoCorrelation_particle(i, positions, box_size, size):
    #for each particle i calculate min_image_convention(positions[1:size, i] - positions[0:size-1, i], b)
    direction_vector=min_image_convention(positions[1:, i] - positions[0:-1, i], box_size)
    dvnorm = np.linalg.norm(direction_vector, axis=1)
    direction_vector = direction_vector/dvnorm[:, np.newaxis]
    result = []
    # Create a nested for-loop running through each particle and gap values
    for j in range(size - 1):
        result.append(0)
        # data[k + 1][i] - data[k][i]
        d1 = direction_vector[0:-j-2]
        # data[k + j + 2][i] - data[k + j + 1][i]
        d2 = direction_vector[j:-2]

        dot_product = np.sum(d1 * d2, axis=1)
        result[j] = np.sum(dot_product) / (size - j - 1)
    return result
def autoCorrelation(particle_data, box_size, data_compression, time_step, skip):
    # Extracting x and y coordinates from the array
    positions = particle_data[::skip, :, :2]
    size=positions.shape[0]
    num_particles=positions.shape[1]
    # Create an empty list to store the "autoCorrelation list".
    autoCorrelationPerParticle = np.zeros((num_particles, size - 1))

    # Create a nested for-loop running through each particle and gap values
    autoCorrelationPerParticle = np.zeros((num_particles, size))
    autoCorrelationPerParticle=Parallel(n_jobs=-1)(delayed(autoCorrelation_particle)(i, positions, box_size, size) for i in tqdm(range(num_particles)))
    autoCorrelationPerParticle = np.array(autoCorrelationPerParticle)
    
    averageAutoCorrelationPerGap=[np.mean(autoCorrelationPerParticle[:, j]) for j in range(size-1)]
    stdAutoCorrelationPerGap=[np.std(autoCorrelationPerParticle[:, j]) for j in range(size-1)]

    x=[i*data_compression*time_step*skip for i in range(math.floor(size-2))]
    y=averageAutoCorrelationPerGap[:len(x)]
    dy=stdAutoCorrelationPerGap[:len(x)]
    return x, y, dy

x, y, dy=autoCorrelation(particleData, boxSize, dataCompression, timestep, 5)

In [ ]:
def plot_AutoCorrelation(x,y, dy, theta, path):
    # Calculate the Fourier transform of the autoCorrelation data and Find the peaks in the Fourier transform
    fft = np.fft.fft(x)
    freq = np.fft.fftfreq(len(x), d=(dataCompression*timestep))
    peaks, properties = find_peaks(np.abs(fft), prominence=(10.0, None))
    print(2*math.pi/freq[peaks])

    # fit the curve with a sinusoidal decaying exponential
    def func(p, a, b, c, d):
        return  a*np.cos(2*math.pi * (np.array(p)+c)/b)*np.exp(- (np.array(p)+c)/d)
    # popt, pcov = curve_fit(func, x, y, p0=[1.0, 100, 1.0,100])

    # plt.plot(x, y, label=f'Vission Angle \u03B8={round(theta)}\u00B0')
    plt.errorbar(x,y, yerr=dy, ecolor = 'lightblue', capsize=0, label=f'Vission Angle \u03B8={round(theta)}\u00B0')
    # plt.plot(x, func (x, *popt), label=f'Fit:'" a="+str(round(popt[0], 2))+" b="+str(round(popt[1], 2))+" c="+str(round(popt[2], 2)), color='red')
    plt.grid(True)
    plt.xlabel("Time Gap")
    plt.ylabel("autoCorrelation")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=7)

    # Set the figure size (adjust these values as needed)
    fig = plt.gcf()
    fig.set_size_inches(10, 6)
    plt.savefig(path+"/autoCorrelation.png", bbox_inches = 'tight')
    plt.show()
    plt.close()

plot_AutoCorrelation(x,y, dy, theta, path)


# Persistence

In [ ]:
def persistence_particle(i, positions, box_size, size):
    #for each particle i calculate min_image_convention(positions[1:size, i] - positions[0:size-1, i], b)
    direction_vector=min_image_convention(positions[1:, i] - positions[0:-1, i], box_size)
    dvnorm = np.linalg.norm(direction_vector, axis=1)
    direction_vector = direction_vector/dvnorm[:, np.newaxis]
    cosine_matrix = np.dot(direction_vector, direction_vector.T)
    result = []
    # Create a nested for-loop running through each particle and gap values
    for j in range(size):
        result.append(0)
        for k in range(size - j - 2):
            cosines = cosine_matrix[k,k:k+j+1]                 #Broadcasting of numpy arrays is the reason we can do the next step
            flag = np.any(cosines < 0)
            if not flag: result[j] += 1.0
        result[j] = result[j]/ int(size - j)
    return result

def persistence(particle_data, box_size, data_compression, time_step, skip):
    # Extracting x and y coordinates from the array
    positions = particle_data[::skip, :, :2]
    size=positions.shape[0]
    num_particles=positions.shape[1]
    
    persistencePerParticle = np.zeros((num_particles, size))
    persistencePerParticle=Parallel(n_jobs=-1)(delayed(persistence_particle)(i, positions, box_size, size) for i in tqdm(range(num_particles)))
    persistencePerParticle = np.array(persistencePerParticle)
    # Create a for loop running through persistencePerParticle[i] and finding the mean persistence
    averagePersistencePerGap=[np.mean(persistencePerParticle[:, j]) for j in range(size)]

    x = [i*data_compression*time_step*skip for i in range(size-1)]
    y = averagePersistencePerGap[:len(x)]
    
    return x, y
x, y=persistence(particleData, boxSize, dataCompression, timestep, 5)

In [ ]:
def plot_persistence(x, y, theta, save_path):
    color = cmap(theta/90)  # Use theta to get a unique color from the colormap
    theta_marker = marker_shapes.get(round(theta), 'o')  # Default to circle if theta not in the dictionary
    #fit and plot the data with an exponential function and print the fit values
    def func(v, a, b, c):
        return a * np.exp(-np.array(v)/b) + c
    popt, pcov = curve_fit(func, x, y, p0=[1, 100, 0])
    # plt.plot(x, func(x, *popt), label=f'Fit:'" a="+str(round(popt[0], 2))+" b="+str(round(popt[1], 2))+" c="+str(round(popt[2], 2)), color=color)
    print (popt)

    plt.plot(x, y, label=f'Vission Angle \u03B8={round(theta)}\u00B0', marker=theta_marker, color=color, markersize=2)
    # plt.scatter(x, y, label=f'Vission Angle \u03B8={round(theta)}\u00B0', marker=theta_marker, color=color, s=2)
    
    # plt.xscale('log')
    # plt.yscale('log', base=2)
    plt.grid(True)
    plt.xlabel("Time Gap")
    plt.ylabel("Persistence")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=7)
    # Set the figure size (adjust these values as needed)
    fig = plt.gcf()
    fig.set_size_inches(10, 6)
    plt.savefig(save_path+"/persistence.png", bbox_inches = 'tight')
    return popt
plot_persistence(x, y, theta, path)

In [ ]:
def persistence(datain):
    data=datain
    size=particleData.shape[0]
    # Create an empty list to store the "Persistence list" and  average persistence per gap
    persistencePerParticle = [[0.0] * (size) for i in range(numParticles)]
    averagePersistencePerGap = [0.0] * (size)
    # Create a nested for-loop running through each particle all the possible gap values to create the Persistence Matrix; here j will be called "gap"
    for i in tqdm(range(numParticles)):
        for j in range(size):
            for k in range(size - j - 1):  # -1 because we want to avoid the last particle as there is no k+1 for it
                flag = 0
                d1x = min_image_convention(data[k + 1][i][0] - data[k][i][0], boxSize)
                d1y = min_image_convention(data[k + 1][i][1] - data[k][i][1], boxSize)
                d1norm = math.sqrt(d1x**2 + d1y**2)
                for l in range(j + 1):
                    d2x = min_image_convention(data[k + l + 1][i][0] - data[k + l][i][0], boxSize)
                    d2y = min_image_convention(data[k + l + 1][i][1] - data[k + l][i][1], boxSize)
                    d2norm = math.sqrt(d2x**2 + d2y**2)
                    if (d1x * d2x + d1y * d2y) / (d1norm * d2norm) < 0:  # if cos becomes negative
                        flag = 1
                        break
                if flag == 0:
                    persistencePerParticle[i][j] += 1.0

            persistencePerParticle[i][j] /= int(size - j)

    # Create a for loop running through persistencePerParticle[i] and finding the mean persistence
    for j in tqdm(range(size)):
        persistenceValues = []
        for i in range(numParticles):
            # Add the persistence of each particle at a given gap to the list
            persistenceValues.append(persistencePerParticle[i][j])

        # Calculate the mean  of the persistence values
        averagePersistencePerGap[j] = np.mean(persistenceValues)

    x = [i*dataCompression*timestep for i in range(size-1)]
    y = averagePersistencePerGap[:len(x)]
    
    return y, x
x, y=persistence(data)

# Simultaneous Averaging Plots

In [ ]:
parent_folder = r"Runs\High Density\CRun" 
file_paths = [f.path+"/" for f in os.scandir(parent_folder) if f.is_dir()]

In [ ]:
def simPlot(file_paths):
    Plot_Data={}
    No_of_Runs_per_theta={}
    for file_path in tqdm(file_paths):
        data = load_data(file_path)
        particle_data = np.array(data)
        
        variable_dict=save_param(file_path)
        theta=(float(variable_dict["theta"]))
        boxSize=float(variable_dict["boxSize"])
        timestep=float(variable_dict["timestep"])
        dataCompression=int(variable_dict["dataCompression"])
        
        # a = np.array(MSD(particle_data[::5], boxSize, timestep, dataCompression, 1))
        a = np.array(persistence(particle_data, boxSize, dataCompression, timestep, 5))

        if theta not in Plot_Data:
            Plot_Data[theta] = a
            No_of_Runs_per_theta[theta] = 1
        else:
            Plot_Data[theta][1] += a[1]
            No_of_Runs_per_theta[theta] += 1
    print (No_of_Runs_per_theta)

    for theta in Plot_Data: Plot_Data[theta][1] /= No_of_Runs_per_theta[theta]
    return Plot_Data
l=simPlot(file_paths)
l = dict(sorted(l.items()))

In [ ]:
fit_values={}
for theta in l:
        popt=plot_persistence(l[theta][0], l[theta][1], theta, parent_folder)
        # popt = plot_MSD(l[theta][0], l[theta][1], theta, parent_folder)
        fit_values[theta]=popt
# Save to Excel
df = pd.DataFrame.from_dict(fit_values, orient='index')
# df.to_excel(parent_folder+"/fit_valuesMSD.xlsx")
df.to_excel(parent_folder+"/fit_valuesPersistence.xlsx")

In [ ]:
# Read from Excel and save it as a dictionary
df = pd.read_excel(parent_folder+"/fit_valuesPersistence.xlsx")
fit_values = df.set_index('Unnamed: 0').T.to_dict('list')
#plot the fit values
plt.plot(list(fit_values.keys()), [i[1] for i in list(fit_values.values())], label='b', marker="s", markersize=2)
plt.xlabel("Vission Angle")
plt.ylabel("Fit Values")
plt.legend()
plt.savefig(parent_folder+"/Persistencefit_values.png")
# plt.savefig(parent_folder+"/MSDfit_values.png")
plt.show()

In [ ]:
def simGif():
    parent_folder = r"Runs\High Density\CRun" 
    file_paths = [f.path+"/" for f in os.scandir(parent_folder) if f.is_dir()]
    for file_path in tqdm(file_paths):
        data = load_data(file_path)
        particleData = np.array(data)
        
        variable_dict=save_param(file_path)
        theta=(float(variable_dict["theta"]))
        boxSize=float(variable_dict["boxSize"])
        timestep=float(variable_dict["timestep"])
        dataCompression=int(variable_dict["dataCompression"])
        animator(particleData, boxSize, file_path, 5)
simGif()

In [ ]:
def simAutoC():
    parent_folder = r"Runs\High Density\CRun" 
    file_paths = [f.path+"/" for f in os.scandir(parent_folder) if f.is_dir()]
    for file_path in tqdm(file_paths):
        data = load_data(file_path)
        particleData = np.array(data)
        
        variable_dict=save_param(file_path)
        theta=(float(variable_dict["theta"]))
        boxSize=float(variable_dict["boxSize"])
        timestep=float(variable_dict["timestep"])
        dataCompression=int(variable_dict["dataCompression"])
        out=autoCorrelation(particleData, boxSize, dataCompression, timestep, 5)
        plot_AutoCorrelation(out[0],out[1], out[2], theta, file_path)
        
simAutoC()

In [ ]:
cProfile.run("persistence(data)")